In [0]:
loan_df = (spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv("/Volumes/workspace/default/workshop/loan_applications.csv")
)
trans_df = (spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv("/Volumes/workspace/default/workshop/transactions.csv")
)

In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS bronze")

In [0]:
loan_df.write.mode("overwrite").format("delta").saveAsTable("bronze.loan_applications")
trans_df.write.mode("overwrite").format("delta").saveAsTable("bronze.transactions")

In [0]:
# ===================================
# 2️⃣  Limpieza & Calidad de Datos - Silver Layer
# ===================================
from pyspark.sql.functions import col, when, isnan, count, avg, sum as spark_sum
from pyspark.sql.types import IntegerType, DoubleType
# Cargar desde Bronze
loan_df = spark.table("bronze.loan_applications")
trans_df = spark.table("bronze.transactions")

# Ejemplo de validación: eliminar nulos críticos y convertir tipos
loan_silver = (
    loan_df
    .dropna(subset=["customer_id", "loan_amount_requested"])
    .withColumn("loan_amount_requested", col("loan_amount_requested").cast(DoubleType()))
    .withColumn("fraud_flag", col("fraud_flag").cast(IntegerType()))
)

trans_silver = (
    trans_df
    .dropna(subset=["customer_id", "transaction_amount"])
    .withColumn("transaction_amount", col("transaction_amount").cast(DoubleType()))
    .withColumn("fraud_flag", col("fraud_flag").cast(IntegerType()))
)


In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS silver")

In [0]:
# Calidad: Filtrar registros con montos negativos, etc.
loan_silver = loan_silver.filter(col("loan_amount_requested") > 0)
trans_silver = trans_silver.filter(col("transaction_amount") > 0)

loan_silver.write.mode("overwrite").format("delta").saveAsTable("silver.loan_applications")
trans_silver.write.mode("overwrite").format("delta").saveAsTable("silver.transactions")

In [0]:
# ===================================
# 3️⃣  Enriquecimiento & KPIs - Gold Layer
# ===================================

from pyspark.sql.functions import col

# Renombrar columnas antes de la unión para evitar conflictos
loan_silver = loan_silver.withColumnRenamed("fraud_flag", "loan_fraud_flag")
trans_silver = trans_silver.withColumnRenamed("fraud_flag", "trans_fraud_flag")


In [0]:
# Unir loan y transactions para perfil de cliente
joined_df = loan_silver.join(trans_silver, on="customer_id", how="inner")

display(joined_df)

In [0]:
joined_df.display()

In [0]:
from pyspark.sql.functions import avg, sum, count, col

gold_df = (
    joined_df.groupBy("customer_id")
    .agg(
        avg("loan_amount_requested").alias("avg_loan_amount"),
        avg("transaction_amount").alias("avg_transaction_amount"),
        sum(col("loan_fraud_flag").cast("int")).alias("num_frauds"),
        count("*").alias("num_transactions")
    )
)
display(gold_df)

In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS gold")

In [0]:
# KPI: Porcentaje de fraude por cliente
gold_df = gold_df.withColumn(
    "fraud_ratio",
    col("num_frauds") / col("num_transactions")
)

gold_df.write.mode("overwrite").format("delta").saveAsTable("gold.customer_profile")

In [0]:
# Dataset final para entrenar modelo de fraude:
# Etiqueta: si el cliente tuvo alguna actividad fraudulenta
ml_input_df = gold_df.withColumn(
    "is_fraudulent",
    when(col("fraud_ratio") > 0, 1).otherwise(0)
)

ml_input_df.write.mode("overwrite").format("delta").saveAsTable("gold.ml_input_fraud_detection")

In [0]:
# ===================================
# 1️⃣  Cargar Silver - Tablas extra
# ===================================
loan_silver = spark.table("silver.loan_applications")
trans_silver = spark.table("silver.transactions")

In [0]:
from pyspark.sql.functions import (
    col, avg, sum as spark_sum, count, countDistinct, stddev, max as spark_max, min as spark_min, expr, when
)

In [0]:

# ===================================
# 2️⃣  KPI Loan Level
# ===================================
loan_kpis = (
    loan_silver.groupBy("customer_id")
    .agg(
        count("*").alias("num_loan_applications"),
        avg("loan_amount_requested").alias("avg_loan_amount"),
        stddev("loan_amount_requested").alias("stddev_loan_amount"),
        spark_max("loan_amount_requested").alias("max_loan_amount"),
        spark_min("loan_amount_requested").alias("min_loan_amount"),
        avg("interest_rate_offered").alias("avg_interest_rate"),
        avg("cibil_score").alias("avg_cibil_score"),
        avg("debt_to_income_ratio").alias("avg_dti"),
        spark_sum("fraud_flag").alias("loan_fraud_count")
    )
)

In [0]:
# ===================================
# 3️⃣  KPI Transaction Level
# ===================================
trans_kpis = (
    trans_silver.groupBy("customer_id")
    .agg(
        count("*").alias("num_transactions"),
        countDistinct("merchant_name").alias("num_unique_merchants"),
        avg("transaction_amount").alias("avg_transaction_amount"),
        stddev("transaction_amount").alias("stddev_transaction_amount"),
        spark_max("transaction_amount").alias("max_transaction_amount"),
        spark_min("transaction_amount").alias("min_transaction_amount"),
        spark_sum(when(col("is_international_transaction") == 1, 1).otherwise(0)).alias("num_international_txn"),
        spark_sum("fraud_flag").alias("transaction_fraud_count")
    )
)

In [0]:
# ===================================
# 4️⃣  Enriquecimiento: Ratio, Flags
# ===================================

# Join de Loan y Transaction KPIs
gold_df_2 = (
    loan_kpis.join(trans_kpis, on="customer_id", how="outer")
    .fillna(0)  # Llenar nulos por clientes sin transacciones o sin préstamos
)


In [0]:
# Feature Engineering
gold_df_2 = (
    gold_df_2.withColumn("loan_to_txn_ratio", 
                       col("num_loan_applications") / (col("num_transactions") + expr("1e-9")))
    .withColumn("avg_txn_to_avg_loan_ratio",
                col("avg_transaction_amount") / (col("avg_loan_amount") + expr("1e-9")))
    .withColumn("overall_fraud_count", 
                col("loan_fraud_count") + col("transaction_fraud_count"))
    .withColumn("has_fraud", when(col("overall_fraud_count") > 0, 1).otherwise(0))
)

In [0]:
# ===================================
# 5️⃣  Guardar Gold Optimizado
# ===================================

# Mejora de rendimiento: particionar por etiqueta de fraude
gold_df_2.write.mode("overwrite").format("delta") \
    .partitionBy("has_fraud") \
    .saveAsTable("gold.customer_fraud_profile")